In [ ]:
# Aim Tracker

import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils
face_mesh = mp_face.FaceMesh(refine_landmarks=True)
mp_hand = mp.solutions.hands
hand = mp_hand.Hands(min_detection_confidence=0.9,min_tracking_confidence=0.9)


FORHEAD_IDX = 10

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = face_mesh.process(rgb)
    h,w,_ = f.shape
    if res.multi_face_landmarks:
        for face_marks in res.multi_face_landmarks:
            forehead = face_marks.landmark[FORHEAD_IDX]
            x = int(forehead.x*w)
            y = int(forehead.y*h)

            cv2.circle(f,(x,y),10,(0,0,255),3)
            cv2.line(f,(x-20,y),(x+20,y),(0,0,255),2)
            cv2.line(f,(x,y-20),(x,y+20),(0,0,255),2)

            cv2.putText(f,f"Target: ({x},{y})",(x+15,y-15),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0),1)

    res1 = hand.process(rgb)
    if res1.multi_hand_landmarks:
        for landmarks in res1.multi_hand_landmarks:
            mp_draw.draw_landmarks(f,landmarks,mp_hand.HAND_CONNECTIONS)

    cv2.imshow("Aim tracker",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()